In [1]:
import pandas as pd

In [2]:
headers = pd.read_csv('../Data/handles-data.csv')
tweets = pd.read_csv('../Data/tweets-data.csv')
union = pd.read_csv('../Data/union.csv', sep=";")
pd.options.display.float_format = '{:.2f}'.format

/home/julianmelero/twitter-ai/twai/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/julianmelero/twitter-ai/twai/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Target

El target de nuestro proyecto va a ser el campo **nretweets** (número de Retweets).

He decidido usar este campo por su posible valor para conocer si el tuit va a tener la repercusión deseada o no.

He llegado a este conclusión después de ver cómo se comporta dependiendo de la hora y del día de la semana. Aunque no existen variaciones muy grandes, sí que he podido comprobar que ciertas horas y ciertos días hay más inteacción dentro de la red social.
Donde se puede ver que los usuarios dan más a me gusta y retuitean. Por lo que seguramente exista un patrón en los datos, dado un patrón de conducta humana. Por ejemplo, en las horas de "sueño" existe menos interacción.



# Limpieza de los Datos

## Eliminar No interesantes

In [3]:
print(tweets.columns.tolist())

['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place', 'tweet', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str', 'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video', 'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url', 'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src', 'trans_dest']


In [83]:
# datosInteresantes = union[['day','hour','retweet','nlikes']]
datosInteresantes = union[['day','hour']]
df_c = datosInteresantes.copy()
target = union[['nretweets']]
print(datosInteresantes.count())
print(target.count())

day     40867
hour    40867
dtype: int64
nretweets    40867
dtype: int64


### Nulos

In [5]:
datosInteresantes.isnull().sum()

day         0
hour        0
retweet     0
nreplies    0
nlikes      0
dtype: int64

### Separamos datos categóricos

In [6]:
datosInteresantes.dtypes

day         int64
hour        int64
retweet      bool
nreplies    int64
nlikes      int64
dtype: object

El campo retweet es categórico (Verdadero o Falso)

In [7]:
datosInteresantes['retweet']

0         True
1        False
2         True
3        False
4        False
         ...  
40862    False
40863    False
40864    False
40865    False
40866    False
Name: retweet, Length: 40867, dtype: bool

In [8]:
from sklearn.preprocessing import OneHotEncoder

In [36]:
df_c = datosInteresantes.copy(deep = True)
enc_ohe = OneHotEncoder()
enc_ohe.fit(df_c[["retweet"]])

OneHotEncoder()

In [37]:
data_encoder = enc_ohe.transform(df_c[['retweet']]).toarray()
print(data_encoder)

[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


Lo unimos al dataset

In [38]:
df_encoder = pd.DataFrame(data_encoder)
df_encoder.head()
df_c = df_c.join(df_encoder,lsuffix='_left', rsuffix='_right')
df_c.drop(['retweet'], axis=1, inplace=True)


In [79]:
df_c

,day,hour,nlikes
0,4,0,0
1,2,22,5266
2,2,18,0
3,2,18,5638
4,2,7,8810
...,...,...,...
40862,2,13,961
40863,2,13,224
40864,2,13,240
40865,2,13,1357


## Separar datos Entrenamiento

In [13]:
from sklearn.model_selection import train_test_split

In [84]:
X_train, X_test, y_train, y_test = train_test_split(df_c, target, test_size=0.3)
print(X_train.shape)
print(X_test.shape)


(28606, 2)
(12261, 2)


## Nulos

In [15]:
X_train.isnull().sum()

day         0
hour        0
nreplies    0
nlikes      0
0           0
1           0
dtype: int64

# Entrenar Modelo

In [16]:

import numpy as np
from sklearn.linear_model import LinearRegression

In [74]:
np.random.seed(5)


In [42]:
model=LinearRegression()


In [19]:
# X_train = np.asarray(X_train).astype(np.int64)

In [85]:
model.fit(X_train, y_train) 

LinearRegression()

In [86]:

predictions = model.predict(X_test)
X_tested = X_test.copy()
print(X_tested.shape)
print(predictions.shape)
X_tested['predictions'] = predictions
X_tested['Target_test'] = y_test

# X_tested['difference'] = (y_test - X_tested['predictions'])
X_tested
# X_test['predictions'] = predictions
# X_test['defference'] = (y_test) - (X_test.predictions)

# día,hora,nreplies,nlikes,0_retweet,1_retweet
# print(model.predict([[6,3,0,0,1,0]]))

(12261, 2)
(12261, 1)


,day,hour,predictions,Target_test
27856,1,18,285.74,33
18060,3,23,248.94,2
270,1,15,295.21,537
30676,7,8,254.27,42
33448,5,9,272.12,9
...,...,...,...,...
11147,3,6,302.61,11
39368,1,20,279.42,1015
18293,5,19,240.55,28
29546,7,19,219.53,0


In [70]:
print(model.predict([[7,15,1,0]]))
X_tested.describe()

[[190.43616088]]


,day,hour,0,1,predictions,Target_test
count,12261.00,12261.00,12261.00,12261.00,12261.00,12261.00
mean,3.83,13.06,0.85,0.15,300.54,257.90
std,1.90,7.35,0.36,0.36,124.55,1545.44
min,1.00,0.00,0.00,0.00,157.80,0.00
25%,2.00,6.00,1.00,0.00,227.53,4.00
50%,4.00,15.00,1.00,0.00,260.90,16.00
75%,5.00,19.00,1.00,0.00,302.81,76.00
max,7.00,23.00,1.00,1.00,679.17,77827.00
